In [1]:
import os
import sys
import re
import copy
import pandas as pd
from tqdm import tqdm
from selenium import webdriver

# set utils
util_path = os.pardir
if util_path not in sys.path:
    sys.path.append(os.pardir)
from utils.helper import load_config
from crawler import Crawler, Utills

# load config file
Project_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
CONFIG_PATH = "config.yaml"
config = load_config(os.path.join(Project_PATH, 'jobkorea', CONFIG_PATH))
init_url = config['init_url']
selen_path_dict = config['selen_path']
solutions = config['solution']
platform_string = config['platform']

# chrome option
chrome_opt = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_opt.add_experimental_option("prefs", prefs) 
chrome_opt.add_experimental_option("detach", True) # prevent automatically closed tab
chrome_opt.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_opt.headless = True # it should be run without UI (Headless)


#location
location = "home"


if location == 'home':
    crawler = Crawler(selen_path = selen_path_dict['home'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = solutions,
                        platform_string = platform_string)
elif location == 'laptop':
    crawler = Crawler(selen_path = selen_path_dict['laptop'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = solutions,
                        platform_string = platform_string)
elif location == 'khrrc':
    crawler = Crawler(selen_path = selen_path_dict['khrrc'],
                        chromeOption=chrome_opt,
                        init_url=init_url, 
                        solutions = solutions,
                        platform_string = platform_string)
else:
    raise NotImplementedError

In [3]:
#unlike other notebook, raw_data is previously crawled dataframe which contained error information
raw_data = pd.read_excel(os.path.join(Project_PATH, 'jobkorea', 'result', 'cleaned_230427.xlsx'), engine='openpyxl')
raw_data.shape

(11185, 15)

# Platform

In [4]:
for idx, row in tqdm(raw_data.iterrows()):
    if (row['사용솔루션'] == 'error') and (row['채널톡사용여부'] == 'error'):        
            raw_data.loc[idx, '플랫폼입점여부'] = 'error'
    else:
        if idx < 5:
            shopping_mall_url = row['도메인명']
            rs = crawler.search_platform(shopping_mall_url)
            raw_data.loc[idx, '플랫폼입점여부'] = rs['플랫폼입점여부']
        else:
            break

raw_data.to_excel(os.path.join(Project_PATH, 'jobkorea', 'result', 'rs_shopping_mall_platfrom.xlsx'))

5it [00:02,  1.78it/s]
